# Trace Agent Creation and Invocation
This notebook illustrates how to emit OpenTelemetry spans for agent workflows using the [GenAI semantic conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/).

We simulate creating the Cora retail agent and invoking it through Azure AI Foundry so you can see which attributes to populate for `create_agent`, `invoke_agent`, and nested tool execution spans.

## 1. Configure a tracer
Use the OpenTelemetry SDK with a console exporter so you can inspect the span payload before sending it to a backend.

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from opentelemetry.trace import SpanKind
import json

provider = TracerProvider()
trace.set_tracer_provider(provider)
console_exporter = ConsoleSpanExporter()
provider.add_span_processor(SimpleSpanProcessor(console_exporter))

tracer = trace.get_tracer("labs.5.observability.agent")
print("Tracer ready — spans will be written to stdout.")

## 2. Emit spans that follow the GenAI conventions
Populate the required and recommended attributes for the agent lifecycle.
- `create_agent` spans describe provisioning remote agents.
- `invoke_agent` spans describe the conversational request/response.
- Nested `execute_tool` spans show how tool calls can be linked into the trace.

In [ ]:
conversation_id = "retail-session-001"
system_prompt = (
    "You are Cora, a polite, factual, and helpful Zava retail assistant. "
    "Answer with concise, markdown-friendly responses."
)
customer_prompt = "What's the best primer for bathroom cabinets?"
agent_reply = (
    "🎨 Great choice! Zava GripLock Primer at $24 seals moisture. "
    "Need brushes too?"
)

with tracer.start_as_current_span("create_agent cora-retail-agent", kind=SpanKind.CLIENT) as span:
    span.set_attribute("gen_ai.provider.name", "azure.ai.inference")
    span.set_attribute("gen_ai.operation.name", "create_agent")
    span.set_attribute("gen_ai.agent.name", "cora-retail-agent")
    span.set_attribute("gen_ai.agent.id", "agents/cora-retail-agent")
    span.set_attribute("gen_ai.agent.description", "Cora retail assistant for Zava DIY customers.")
    span.set_attribute("gen_ai.request.model", "gpt-4o-mini")
    span.set_attribute("gen_ai.system_instructions", system_prompt)
    span.set_attribute("server.address", "cora-agents.eastus2.inference.ai.azure.com")
    span.set_attribute("server.port", 443)
    span.set_attribute("azure.resource_provider.namespace", "Microsoft.CognitiveServices")

with tracer.start_as_current_span("invoke_agent cora-retail-agent", kind=SpanKind.CLIENT) as span:
    span.set_attribute("gen_ai.provider.name", "azure.ai.inference")
    span.set_attribute("gen_ai.operation.name", "invoke_agent")
    span.set_attribute("gen_ai.agent.name", "cora-retail-agent")
    span.set_attribute("gen_ai.agent.id", "agents/cora-retail-agent")
    span.set_attribute("gen_ai.request.model", "gpt-4o-mini")
    span.set_attribute("gen_ai.request.max_tokens", 400)
    span.set_attribute("gen_ai.request.temperature", 0.2)
    span.set_attribute("gen_ai.request.top_p", 0.95)
    span.set_attribute("gen_ai.request.frequency_penalty", 0.0)
    span.set_attribute("gen_ai.request.presence_penalty", 0.0)
    span.set_attribute("gen_ai.response.model", "gpt-4o-mini")
    span.set_attribute("gen_ai.response.finish_reasons", ["stop"])
    span.set_attribute("gen_ai.response.id", "resp-8b5c")
    span.set_attribute("gen_ai.usage.input_tokens", 152)
    span.set_attribute("gen_ai.usage.output_tokens", 38)
    span.set_attribute("gen_ai.conversation.id", conversation_id)
    span.set_attribute("gen_ai.system_instructions", system_prompt)
    span.set_attribute("gen_ai.input.messages", json.dumps([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": customer_prompt}
    ], ensure_ascii=False))
    span.set_attribute("gen_ai.output.messages", json.dumps([
        {"role": "assistant", "content": agent_reply}
    ], ensure_ascii=False))
    span.set_attribute("server.address", "cora-agents.eastus2.inference.ai.azure.com")
    span.set_attribute("server.port", 443)

    with tracer.start_as_current_span("execute_tool product_search", kind=SpanKind.INTERNAL) as tool_span:
        tool_span.set_attribute("gen_ai.operation.name", "execute_tool")
        tool_span.set_attribute("gen_ai.tool.name", "product_search")
        tool_span.set_attribute("gen_ai.tool.type", "function")
        tool_span.set_attribute("gen_ai.tool.call.id", "call-42")
        tool_span.set_attribute("gen_ai.tool.call.arguments", json.dumps({
            "sku": "PRIMER-4821",
            "query": customer_prompt
        }))
        tool_span.set_attribute("gen_ai.tool.call.result", json.dumps({
            "name": "Zava GripLock Primer",
            "price": 24.0,
            "available_stock": 67
        }))

print("Done — inspect the console output to see each span and its attributes.")